In [1]:
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data
import time

tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input placeholders
X = tf.placeholder(tf.float32, shape=[None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, shape=[None, 10])

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv     -> (?, 14, 14, 64)
#    Pool     -> (?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])

# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

start_time = time.time()

# train model
print('Learning started, It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    start_time_epoch = time.time()
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost), " %.2f seconds" % (time.time() - start_time_epoch))

print('Learning Finished!')
print("--- %.2f seconds ---" %(time.time() - start_time))

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started, It takes sometime.
Epoch: 0001 cost =  0.345577185  38.61 seconds
Epoch: 0002 cost =  0.091736604  37.80 seconds
Epoch: 0003 cost =  0.068284046  37.66 seconds
Epoch: 0004 cost =  0.056339833  37.70 seconds
Epoch: 0005 cost =  0.047010720  37.75 seconds
Epoch: 0006 cost =  0.041194586  37.86 seconds
Epoch: 0007 cost =  0.036663712  37.51 seconds
Epoch: 0008 cost =  0.032757639  37.48 seconds
Epoch: 0009 cost =  0.027963868  37.50 seconds
Epoch: 0010 cost =  0.025047483  37.40 seconds
Epoch: 0011 cost =  0.022065875  38.03 seconds
Epoch: 0012 cost =  0.020263703  39.23 seconds
Epoch: 0013 cost =  0.016754853  39.05 seconds
Epoch: 0014 cost =  0.015507657  39.05 seconds
Epoch: 0015 cost =  0.013157484  39.12 seconds
Learning Finished!
--- 571.75 seconds ---
Accuracy: 0.9883
